In [1]:
#%load_ext autoreload
#%autoreload 2
%matplotlib inline
%cd '../'

/Users/lolo/quantic/science/HomeHots/HOTS_clone_laurent


In [2]:

import numpy as np
import matplotlib.pyplot as plt
import os
import time
timestr = '20201021'
    

### Building matrix for logistic regression

In [ ]:
#fname = 'Records/EXP_03_NMNIST/'+timestr+'_hots_vectorzLR.pkl'
fname = '/tmp/'+timestr+'_hots_vectorzLR_2'

if not os.path.isfile(fname + '_X.npy'):
    
    import pickle
    from HOTS.ToolsMonitor import GenerateActivationMap, DisplayActivationMap
    from HOTS.Event import Event, SimpleAlphabet, LoadNMNIST
    from HOTS.STS import STS
    from HOTS.Layer import ClusteringLayer
    from HOTS.ToolsMonitor import (
        DisplayImage,
        DisplaySurface3D,
        DisplaySurface2D,
        DisplayConvergence,
    )
    from HOTS.Tools import SaveObject, LoadObject
    from HOTS.Network import Network
    from HOTS.Event import conv2eve

    tau = 9e-4 # -> tau=1ms, si on prend 10 ms on est à 1s pour la dernière couche et les vidéos font 0.3s en moyenne
    R = 2
    filthr = 2
    nbkNN = 3
    algo = 'lagorce'
    decay = 'exponential'
    hom = True
    krnlinit = 'rdn'
    nb_cluster = [4, 8, 16]
    ImageSize = (34, 34)
    DataPath = 'Data/testsetnmnist.p'

    NbClusteringData = 15
    NbTrainingData = 40
    NbTestingData = 40
    
    NbClusteringData = 1500
    NbTrainingData = 5000
    NbTestingData = 2500

    _1ofich = True
    event_tr, event_te, event_cl, label_tr, label_te = LoadNMNIST(
    NbTrainingData, NbTestingData, NbClusteringData, OneOfEach=_1ofich, Path=DataPath, OutOnePolarity=False, ListPolarities=None, verbose=0)
    
    #timestr = time.strftime("%Y%m%d")
    hom = False
    
    if hom==True:
        fname_model = 'Records/EXP_03_NMNIST/'+timestr+'_hots_'+str(tau*1000)+'ms_'+algo+'_homeo.pkl'
    else:
        fname_model = 'Records/EXP_03_NMNIST/'+timestr+'_hots_'+str(tau*1000)+'ms_'+algo+'.pkl'
    print(fname)

    L1 = ClusteringLayer(tau=tau,R=R,verbose=0,ThrFilter=filthr,LearningAlgo=algo,kernel=decay,homeo=hom,init=krnlinit)
    L2 = ClusteringLayer(tau=10 * tau,R=2 * R,verbose=0,ThrFilter=filthr,LearningAlgo=algo,kernel=decay,homeo=hom,init=krnlinit)
    L3 = ClusteringLayer(tau=10 * 10 * tau,R=2 * 2 * R,verbose=0,ThrFilter=filthr,LearningAlgo=algo,kernel=decay,homeo=hom,init=krnlinit)
    Net = Network([L1, L2, L3])
    if not os.path.isfile(fname_model):
        ClusterLayer, event_output = Net.TrainCluster(
                event=event_cl, NbClusterList=nb_cluster, to_record=True, NbCycle=1
            )
        SaveObject(ClusterLayer, fname)
    else: 
        ClusterLayer, Classif0 = LoadObject(fname)
    
    # run the events through the network
    homrun = False
    event0_o_tr2 = Net.RunNetwork(event_tr, NbClusterList=ClusterLayer, homrun=homrun)
    event0_o_te2 = Net.RunNetwork(event_te, NbClusterList=ClusterLayer, homrun=homrun)    
    
    # gather data
    data_len = ImageSize[0]*ImageSize[1]*len(event0_o_tr2.ListPolarities)
    n_events = event0_o_tr2.time.shape[0]
    tau_cla = 1e-2
    sample_events = 200
    sample_space = 4
    
    X = np.zeros((n_events//sample_events, data_len))
    y = np.zeros((n_events//sample_events,))
    data = np.zeros((ImageSize[0]//sample_space, ImageSize[1]//sample_space, len(event0_o_tr2.ListPolarities))) #tmp data

    t_old = 0
    i_label_tr = 0
    for i in range(1, n_events):
        if i == event0_o_te2.ChangeIdx[i_label_tr]:
            i_label_tr += 1

        t_new = event0_o_tr2.time[i]
        data *= np.exp(-(t_new-t_old)/tau_cla)
        data[event0_o_tr2.address[i, 0]//sample_space, event0_o_tr2.address[i,1]//sample_space, event0_o_tr2.polarity[i]] = 1.

        if i % sample_events == 0:
            X[i//sample_events, :] = data.ravel()
            y[i//sample_events] = label_tr[i_label_tr]

    print('Number of events: '+str(X.shape[0])+' - Number of features: '+str(X.shape[1]))
    #SaveObject([X,y], fname)
    np.save(fname + '_X.npy', X)
    np.save(fname + '_y.npy', y)
    
else: 
    #X,y = LoadObject(fname)
    X = np.load(fname + '_X.npy')
    y = np.load(fname + '_y.npy')
    

[2, 5, 9, 1, 3, 6, 7, 0, 8, 4]


In [ ]:
0 % 200


In [ ]:
X.shape

### Performing logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression as LR
lr = LR(random_state=0).fit(X, y)

In [ ]:
lr.score(X, y)

In [ ]:
lr.score(X, y)

In [ ]:
P_pred = lr.predict_proba(X)
P_pred.shape

In [ ]:
plt.plot(np.max(P_pred, axis=1))